# LangChain Expression Language (LCEL)

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple chain

In [3]:
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "bananas"})

"Why did the banana go to the doctor? \n\nBecause it wasn't peeling well! 🍌"

## More complex (add retriever)

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [5]:
vectorstore = DocArrayInMemorySearch.from_texts(
    [
        "Harrison worked at Kensho",
        "Bears like to eat honey",
    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

c:\Users\tchak\anaconda3\envs\langchain\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [6]:
retriever.invoke("Where did Harrison work?")

[Document(metadata={}, page_content='Harrison worked at Kensho'),
 Document(metadata={}, page_content='Bears like to eat honey')]

In [7]:
retriever.invoke("What do bears like to eat?")

[Document(metadata={}, page_content='Bears like to eat honey'),
 Document(metadata={}, page_content='Harrison worked at Kensho')]

In [8]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [9]:
from langchain.schema.runnable import RunnableMap

inputs = RunnableMap({
    "question": lambda x: x["question"],
    "context": lambda x: retriever.invoke(x["question"]),
})
chain = inputs | prompt | model | output_parser

print(inputs.invoke({"question": "Where did Harrison work?"}))

{'question': 'Where did Harrison work?', 'context': [Document(metadata={}, page_content='Harrison worked at Kensho'), Document(metadata={}, page_content='Bears like to eat honey')]}


In [10]:
print(chain.invoke({"question": "Where did Harrison work?"}))

Harrison worked at Kensho.


## Bind (OpenAI functions)

In [11]:
functions = [
    {
        "name": "weather_search",
        "description": "Search for weather given an airport code",
        "parameters": {
            "type": "object",
            "properties": {
                "airport_code": {
                    "type": "string",
                    "description": "The airport code to get the weather for"
                },
            },
            "required": ["airport_code"],
        },
    },
    {
        "name": "sports_search",
        "description": "Search for news of recent sport events",
        "parameters": {
            "type": "object",
            "properties": {
                "team_name": {
                    "type": "string",
                    "description": "The sports team to search for"
                },
            },
            "required": ["team_name"],
        },
    },
]

In [12]:
prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
])
model = ChatOpenAI().bind(functions=functions)

In [13]:
chain = prompt | model
print(chain.invoke({"input": "What's the weather is SF?"}).additional_kwargs)
print(chain.invoke({"input": "How did the Patriots do yesterday?"}).additional_kwargs)

{'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}, 'refusal': None}
{'function_call': {'arguments': '{"team_name":"Patriots"}', 'name': 'sports_search'}, 'refusal': None}


## Fallbacks

In [14]:
from langchain_core.language_models.fake_chat_models import FakeChatModel
import json

In [15]:
simple_model = FakeChatModel()
simple_chain = simple_model | StrOutputParser() |json.loads

challenge = "Write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

print(simple_model.invoke(challenge))

# simple_chain.invoke(challenge) # fails because the output "fake response" can't be converted to JSON

content='fake response' additional_kwargs={} response_metadata={} id='run-20b758e9-6874-40b5-890f-b772940ac5f5-0'


In [16]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'first_line': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'first_line': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'first_line': 'Hope is the thing with feathers that perches in the soul'}}

In [17]:
final_chain = simple_chain.with_fallbacks([chain])
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

## Interface: invoke, batch, stream, async

In [18]:
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [19]:
chain.invoke({"topic": "gorillas"})

'Why did the gorilla go to the doctor? Because he was feeling a little "ape"-y!'

In [20]:
chain.batch([
    {"topic": "fire"},
    {"topic": "horse"},
])

["Why did the firefighter wear red suspenders? To keep his pants up while he's putting out fires!",
 'Why did the horse sit at the computer? He wanted to be a "neigh" sayer!']

In [21]:
for t in chain.stream({"topic": "glass"}):
    print(t)


Why
 did
 the
 glass
 go
 to
 therapy
?
 Because
 it
 had
 a
 lot
 of
 transparency
 issues
!



In [22]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why did the bear break up with his girlfriend?\nBecause he couldn't bear the relationship any longer!"